TODO: 
* define state
* Habla desde el principio de attractores

# Index

* Introduction
    * cite{Wagner, COLOMOTO, GINsim, Boolnet}
    * Define robustness, stability and plasticity
    * GRN perturbations at different levels
    * Biological system: Th17/iTreg
* GRN 
    * Construction (and simplification)
    * Attractors
    * Labeling
* Functions (and topology)
    * Truth table perturbations
    * Overexpression and Knock-Outs
    * Environment
* Transition space and update schema
    * Synchronous vs asynchronous
* Transient perturbations of the state
    * Stochastic noise and directed perturbations
* Appendix
    * Simulation and model checking
    * Importing and exporting with SBML

# Robustness and Platicity in Gene Regulatory Networks

* Organism development, noise and perturbations
* Multiple levels of perturbations
* Common mechanism for robustness and plasticity

* GRN have been used for developmet and plasticity
* Integrate molecular data to predict cell behavior

* Modelling perturbations in GRN and biological implications
* Attractors

Organisms develop in a changing world where they are subject to both intrisic noise and a perturbations of the environment. They need to be both resilient and capable of altering their phenotype depending of the circumstances. This two behaivors coexist in living beings, which indicates that there is a common mechanism that underlies both robustness and plasticity\cite{Wagner?}. 
Gene Regulatory Networks are a usefull tool for studing the robustness and plasticity of biological systems in response to different kinds of perturbations.

## Biological system: Th17 / iTreg network

In this work we will study the Th17/iTreg regulatory network. CD4 + T cells are fundamental for the adaptive immune response. They integrate the signals of the environment and differentiate into different cell types (Th1, Th2, Th17, iTreg, etc), which activate different parts of the inmune system. In particular, Th17 cells have been associated with the inflammation and iTreg cells with the regulation of the inflamatory response.
CD4+ T cells begin as naïve Th0 cells, which do not express a transcription factor. These cells are activated by antigen presentation and differentiate into different cell types depending in the cytokines in the environment. In the presence of IL-6 or IL-21 and TGF$\beta$ Th0 cells differentiate into Th17 cells and express ROR$\gamma$t, IL-21 and IL-17. In the presence of IL-2 and TGF$\beta$ Th0 cells differentiate into iTreg cells and express Foxp3 and TGF$\beta$. Interleukin-10 (IL-10) is a regulatory cytokine that is expressed in multiple cell types, it is activated by various cytokines including IL-6, IL-21 and TGF$\beta$. These cytokines and transcription factors regulate each other and their relationships can be vizualized a as graph.
<img src="minTh17iTreg.png">

## Modeling: Boolean Gene Regulatory Networks

Gene regulatory Networks (GRN)  are deterministic dynamic systems. GRN have been used to study the differentiation, robustness and plasticity of developmental processes in different organisms\cite{}. 
GRN consist of nodes -that can be genes, proteins, biochemical or even biological processes- and edges -that represent the regulatory interactions among the nodes. Using this interactions it is possible to construct functions that describe the state of the nodes, this means, wether the gene or protein is active or inactive. 
The functions of the network are iterated to obtain the stable states or the system or attractors. This attractors correspond to the biological cell types \cite{Kaufman}.
However, as we have alredy discused, biological systems are subjected to noise and perturbations. This perturbations can affect each of the steps
[Fig1 A] \cite{reviewReka?}.

<img src="DiagramGRN.svg">

# Modeling Tool: Gene Regulatory Networks

The interactions of a biological system can be expressed as a set of logical functions using multiple formalisms ([Boolean functions](https://en.wikipedia.org/wiki/Boolean_function), [truth tables](https://en.wikipedia.org/wiki/Truth_table), [Binary Decision Diagrams](https://en.wikipedia.org/wiki/Binary_decision_diagram)).

The number of possible states and the complexity of finding the attractors grows exponentially with the number of nodes. It is possible efficiently find the stable states [without simulations](http://ginsim.org/documentation) or using [model checking](http://dl.acm.org/citation.cfm?id=2014689). However, sometimes it is necessary to [simplify the network](http://ginsim.org/ginsim-doc/current/algo-reduction.html). 

In this tutorial we will use [Boolnet](https://cran.r-project.org/web/packages/BoolNet/index.html) to analyze the network.

## Construction

The Th17/iTreg network can be expressed as a set of boolean rules obtained from the known interactions among the cytokines and transcription factors.

>targets, functions

>IL2, (IL2e | (IL2 &  ! FOXP3)) &  ! (STAT3 | (IL10 & ! FOXP3))

>RORGT, (STAT3 & TGFB) &  ! FOXP3

>STAT3, (IL21e | STAT3 | RORGT) &  ! (IL10 | IL2)

>FOXP3, (IL2 & (TGFB | FOXP3)) &  ! (STAT3 | RORGT)

>TGFB, TGFBe | ((TGFB | FOXP3) &  ! STAT3 )

>IL10, IL10e | (IL10 & (STAT3 | TGFB))

Load BoolNet

In [2]:
#Uncomment next line if you haven't installed BoolNet
#install.packages("BoolNet", repos='http://cran.us.r-project.org')
library(BoolNet)

First, we need to save the rules in a file (Boolnet can only load rules from a file).

In [3]:
fileConn<-file("minTh17iTreg.txt")
writeLines(c(
    "targets, functions",
    "IL2, (IL2e | (IL2 &  ! FOXP3)) &  ! (STAT3 | (IL10 & ! FOXP3))",
    "RORGT, (STAT3 & TGFB) &  ! FOXP3",
    "STAT3, (IL21e | STAT3 | RORGT) &  ! (IL10 | IL2)",
    "FOXP3, (IL2 & (TGFB | FOXP3)) &  ! (STAT3 | RORGT)",
    "TGFB, TGFBe | ((TGFB | FOXP3) &  ! STAT3 )",
    "IL10, IL10e | (IL10 & (STAT3 | TGFB))"
    ), fileConn)
close(fileConn)

Now we can create a network. If you don't specify a node BoolNet will raise a warning and asumme it is an input.
The resulting network should have ten nodes.

In [4]:
net <- loadNetwork("minTh17iTreg.txt")
net

Warning message:
In loadNetwork("minTh17iTreg.txt"): There is no transition function for gene "IL2e"! Assuming an input!Warning message:
In loadNetwork("minTh17iTreg.txt"): There is no transition function for gene "IL21e"! Assuming an input!Warning message:
In loadNetwork("minTh17iTreg.txt"): There is no transition function for gene "TGFBe"! Assuming an input!Warning message:
In loadNetwork("minTh17iTreg.txt"): There is no transition function for gene "IL10e"! Assuming an input!

Boolean network with 10 genes

Involved genes:
IL2 RORGT STAT3 FOXP3 TGFB IL10 IL2e IL21e TGFBe IL10e

Transition functions:
IL2 = (IL2e | (IL2 &  ! FOXP3)) &  ! (STAT3 | (IL10 & ! FOXP3))
RORGT = (STAT3 & TGFB) &  ! FOXP3
STAT3 = (IL21e | STAT3 | RORGT) &  ! (IL10 | IL2)
FOXP3 = (IL2 & (TGFB | FOXP3)) &  ! (STAT3 | RORGT)
TGFB = TGFBe | ((TGFB | FOXP3) &  ! STAT3 )
IL10 = IL10e | (IL10 & (STAT3 | TGFB))
IL2e = IL2e
IL21e = IL21e
TGFBe = TGFBe
IL10e = IL10e

## Attractors and updating

As the state of the network is updated using the functions the network will reach a previously visited state called an attractor. This attractors represent stable states in the dynamics of the network and have been related to cell types or  biological processes like the cell cycle. The set of states that lead to an attractor is called the basin of the attractor [Fig1 C] \cite{Gerherson2004}.

The attractors we obtain can be affected by the update schema (synchronus or asynchronous). We will discuss this later, for now we will use the synchronous attractors.

In [19]:
attr.sync <- getAttractors(net)
attr.sync

Attractor 1 is a simple attractor consisting of 1 state(s) and has a basin of 27 state(s):

 |--<---------|
 V            |
 0000000000   |
 V            |
 |-->---------|


Genes are encoded in the following order: IL2 RORGT STAT3 FOXP3 TGFB IL10 IL2e IL21e TGFBe IL10e

Attractor 2 is a simple attractor consisting of 1 state(s) and has a basin of 2 state(s):

 |--<---------|
 V            |
 1000000000   |
 V            |
 |-->---------|


Genes are encoded in the following order: IL2 RORGT STAT3 FOXP3 TGFB IL10 IL2e IL21e TGFBe IL10e

Attractor 3 is a simple attractor consisting of 1 state(s) and has a basin of 14 state(s):

 |--<---------|
 V            |
 0010000000   |
 V            |
 |-->---------|


Genes are encoded in the following order: IL2 RORGT STAT3 FOXP3 TGFB IL10 IL2e IL21e TGFBe IL10e

Attractor 4 is a simple attractor consisting of 1 state(s) and has a basin of 13 state(s):

 |--<---------|
 V            |
 0000100000   |
 V            |
 |-->---------|


Genes are e

# Transition space and update schema

This attractors where obtained using a synchronous update: the functions where updated concurently (at the same time). However, not all biological processes occur exactly at the same time. We can use asynchronous updates to solve this. A synchronous update is deterministic, this means each state has only one sucessor. However, in asynchoronous updates, states can have more than one sucessors, which can complicate the analysis of the system

In [18]:
attr.sync <- getAttractors(net)
attr.async <- getAttractors(net, type="asynchronous")

Lets compare the number of attractors obtained with the synchronous and asynchronous method

In [6]:
length(attr.sync$attractors)
length(attr.async$attractors)

[1] 58

[1] 30

As you can see, the asynchronous method returns less attractors. Lets compare the two attractors lists. In the first place it is neccesary to obtain a list of the states in each attractor set. 

In [7]:
attr.sync.states <- sapply( attr.sync$attractors, function (a) {a$involvedStates})
attr.async.states <- sapply( attr.async$attractors, function (a) {a$involvedStates})

Lets determine the intersection

In [8]:
length(intersect(attr.sync.states,attr.async.states))

[1] 30

Lets see which states are only present in the synchronous attractors

In [9]:
setdiff(attr.sync.states,attr.async.states)

4
16
89
112
121
129
193
278
345
342
377


Lets see which states are only present in the asynchronous attractors

In [10]:
setdiff(attr.async.states,attr.sync.states)

integer(0)

Synchronous updates are deterministic, this means each state has only one sucessor. However, in asynchoronous updates, states can have more than one sucessor, which makes it hard to determine the basin size. This can complicate the analysis of the system and add computational time. It is for this reason that synchronous updating is usually used.

Asynchronous updates usually result in less attractors, however, all the attractors present in the asynchronous update will be present in the synchronous. It is important to check the asynchronous attractors as some fixed point and cyclic attractors are not robust to the update method and may not occur in biological systems that are asynchronous.

For the following steps we will used the syncronous attractors, as we will use basin sizes.

In [11]:
attr <- attr.sync
attr

Attractor 1 is a simple attractor consisting of 1 state(s) and has a basin of 27 state(s):

 |--<---------|
 V            |
 0000000000   |
 V            |
 |-->---------|


Genes are encoded in the following order: IL2 RORGT STAT3 FOXP3 TGFB IL10 IL2e IL21e TGFBe IL10e

Attractor 2 is a simple attractor consisting of 1 state(s) and has a basin of 2 state(s):

 |--<---------|
 V            |
 1000000000   |
 V            |
 |-->---------|


Genes are encoded in the following order: IL2 RORGT STAT3 FOXP3 TGFB IL10 IL2e IL21e TGFBe IL10e

Attractor 3 is a simple attractor consisting of 1 state(s) and has a basin of 14 state(s):

 |--<---------|
 V            |
 0010000000   |
 V            |
 |-->---------|


Genes are encoded in the following order: IL2 RORGT STAT3 FOXP3 TGFB IL10 IL2e IL21e TGFBe IL10e

Attractor 4 is a simple attractor consisting of 1 state(s) and has a basin of 13 state(s):

 |--<---------|
 V            |
 0000100000   |
 V            |
 |-->---------|


Genes are e

# Labeling

Attractors correspond to cell types. It is very important to verify that all the expected cell types appear in our attractors, if they are not present we might be missing interactions. It is also important to see if there are attractors that do not correspond to known cell types, as they may be predictions or show errors in the construction of the network.

However, when there are a lot of inputs, it is possible that many attractors correspond to a single cell type that can be found in different environments (we will discuss this further). To solve this problem we will label our attractors according to a set of rules.

>labels,  rules

>Th0,   ! (RORGT | FOXP3 | TGFB | IL10)

>Th17,  RORgt & STAT3

>iTreg, FOXP3 & TGFB

>IL10+, IL10 & ! (RORGT | FOXP3 | TGFB)

>TGFB+, TGFB & ! (RORGT | FOXP3 | TGFB)

>IL10+TGFB+, (TGFB & IL10) & ! (RORGT | FOXP3)

It would be useful to have the attractors in 0s and 1s format and know the gene names. Lets make a function for this

In [12]:
dec2binState <- function(x, genes){ 
    state <- as.integer( intToBits(x)[1:length(genes)] )  
    names(state) <- genes
    state
    }

It would also be useful to have a function that labels states. This function will used suposse that the state is in 0s and 1s and that we know the gene names.

In [13]:
labeling.Th17iTreg <- function (state) {
    if (! (state['RORGT'] | state['FOXP3'] | state['TGFB'] | state['IL10']))   return('Th0')
    if (state['RORGT'] & state['STAT3'])  return('Th17')
    if (state['FOXP3'] & state['TGFB'])   return('iTreg')
    if (state['IL10']  & ! (state['RORGT'] | state['FOXP3'] | state['TGFB']))  return('IL10+')
    if (state['TGFB']  & ! (state['RORGT'] | state['FOXP3'] | state['IL10']))  return('TGFB+')
    if (state['IL10']  & state['TGFB'] & ! (state['RORGT'] | state['FOXP3']))  return('IL10+TGFB+')
    else return('NoLabel')
}

Now, lets add a label attribute to our attractors.

In [14]:
for (i in 1:length(attr$attractors))    {
    
    #attr$attractors[[i]]$label <- attr.label[i]
}

attr.states <- sapply( attr$attractors, function (a) {a$involvedStates})
attr.states <- lapply( attr.states, dec2binState, net$genes)
attr.states

Lets label the attractors.

In [15]:
attr.label <- sapply( attr.states, 
})
attr.label


ERROR: Error in parse(text = x, srcfile = src): <text>:2:1: unexpected '}'
1: attr.label <- sapply( attr.states, 
2: }
   ^


Now lets add the label to attr

In [16]:
for (i in 1:length(attr$attractors))    attr$attractors[[i]]$label <- attr.label[i]

ERROR: Error: object 'attr.label' not found
